### AOAI assistants API example - using code interpreter tool! 

In [68]:
import openai
import os
from openai import AzureOpenAI
import json
import pandas as pd
from dotenv import load_dotenv
load_dotenv("credentials.env")

True

In [69]:
import os
import json
from openai import AzureOpenAI
    
# Using different endpoint and API key as Assistants API while writing this code is still in preview and not available in all regions. 
# Hence I have to create AOAI endpoint in East US and use it for creating the assistant.
client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT_EUS"), 
    api_key=os.getenv("AZURE_OPENAI_API_KEY_EUS"),  
    api_version="2024-05-01-preview" #Assistants work in this API
)
# Create an assistant
assistant = client.beta.assistants.create(
    name="Data Visualization",
    instructions=f"You are a helpful AI assistant who makes interesting visualizations based on data." 
    f"You have access to a sandboxed environment for writing and testing code."
    f"When you are asked to create a visualization you should follow these steps:"
    f"1. Write the code."
    f"2. Anytime you write new code display a preview of the code to show your work."
    f"3. Run the code to confirm that it runs."
    f"4. If the code is successful display the visualization."
    f"5. If the code is unsuccessful display the error message and try to revise the code and rerun going through the steps from above again.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4" #You must replace this value with the deployment name for your model.
)

In [70]:
print(assistant.model_dump_json(indent=2))

{
  "id": "asst_3l8W84To9pCPFoCN5YDt82Wb",
  "created_at": 1721848119,
  "description": null,
  "file_ids": null,
  "instructions": "You are a helpful AI assistant who makes interesting visualizations based on data.You have access to a sandboxed environment for writing and testing code.When you are asked to create a visualization you should follow these steps:1. Write the code.2. Anytime you write new code display a preview of the code to show your work.3. Run the code to confirm that it runs.4. If the code is successful display the visualization.5. If the code is unsuccessful display the error message and try to revise the code and rerun going through the steps from above again.",
  "metadata": {},
  "model": "gpt-4",
  "name": "Data Visualization",
  "object": "assistant",
  "tools": [
    {
      "type": "code_interpreter"
    }
  ],
  "top_p": 1.0,
  "temperature": 1.0,
  "tool_resources": {
    "code_interpreter": {
      "file_ids": []
    }
  },
  "response_format": "auto"
}


In [71]:
# Create a thread
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_NO5OUKd5Fj1SYSXUcRgkeXh8', created_at=1721848126, metadata={}, object='thread', tool_resources={})


In [72]:
# Add a user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Create a visualization of a sinewave"
)

In [73]:
thread_messages = client.beta.threads.messages.list(thread.id)
print(thread_messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_eHAeocgMA8hFXz4ebEqLU8er",
      "assistant_id": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "Create a visualization of a sinewave"
          },
          "type": "text"
        }
      ],
      "created_at": 1721848130,
      "file_ids": null,
      "metadata": {},
      "object": "thread.message",
      "role": "user",
      "run_id": null,
      "thread_id": "thread_NO5OUKd5Fj1SYSXUcRgkeXh8",
      "attachments": []
    }
  ],
  "object": "list",
  "first_id": "msg_eHAeocgMA8hFXz4ebEqLU8er",
  "last_id": "msg_eHAeocgMA8hFXz4ebEqLU8er",
  "has_more": false
}


In [86]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  #instructions="New instructions" #You can optionally provide new instructions but these will override the default instructions
)

In [87]:
# Retrieve the status of the run
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

status = run.status
print(status)

in_progress


In [88]:
import time
from IPython.display import clear_output

start_time = time.time()

status = run.status

while status not in ["completed", "cancelled", "expired", "failed"]:
    time.sleep(5)
    run = client.beta.threads.runs.retrieve(thread_id=thread.id,run_id=run.id)
    print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
    status = run.status
    print(f'Status: {status}')
    clear_output(wait=True)

messages = client.beta.threads.messages.list(
  thread_id=thread.id
) 

print(f'Status: {status}')
print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
print(messages.model_dump_json(indent=2))

Status: completed
Elapsed time: 0 minutes 10 seconds
{
  "data": [
    {
      "id": "msg_hjMma69ioUmaCSuJSZXlEJiD",
      "assistant_id": "asst_3l8W84To9pCPFoCN5YDt82Wb",
      "content": [
        {
          "image_file": {
            "file_id": "assistant-xD4sG9BkMGWbf2NZXAo3TqZi"
          },
          "type": "image_file"
        },
        {
          "text": {
            "annotations": [],
            "value": "Here's a visualization of a sine wave generated from the function \\(sin(x)\\) over the range of 0 to 10. The wave oscillates smoothly, demonstrating the classic shape of a sine function."
          },
          "type": "text"
        }
      ],
      "created_at": 1721848262,
      "file_ids": null,
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_EYfWX1fVcxJp6as8gViyMltq",
      "thread_id": "thread_NO5OUKd5Fj1SYSXUcRgkeXh8",
      "attachments": []
    },
    {
      "id": "msg_p5eg4dDV8qiomASRRAiYrbBJ",
      "

In [80]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_p5eg4dDV8qiomASRRAiYrbBJ",
      "assistant_id": "asst_3l8W84To9pCPFoCN5YDt82Wb",
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "It seems there was an issue connecting to the execution environment. Let's try to run the code again to create the visualization of a sine wave."
          },
          "type": "text"
        }
      ],
      "created_at": 1721848157,
      "file_ids": null,
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_MCfNbe0ZYhv8CEfIdXKZLjTp",
      "thread_id": "thread_NO5OUKd5Fj1SYSXUcRgkeXh8",
      "attachments": []
    },
    {
      "id": "msg_eHAeocgMA8hFXz4ebEqLU8er",
      "assistant_id": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "Create a visualization of a sinewave"
          },
          "type": "text"
        }
      ],
      "created_at": 172184813

In [89]:
data = json.loads(messages.model_dump_json(indent=2))  # Load JSON data into a Python object
image_file_id = data['data'][0]['content'][0]['image_file']['file_id']

print(image_file_id)  # Outputs: assistant-1YGVTvNzc2JXajI5JU9F0HMD

assistant-xD4sG9BkMGWbf2NZXAo3TqZi


In [90]:
content = client.files.content(image_file_id)

image= content.write_to_file("data\sinewave.png")

In [ ]:
#Uncomment below code to see sinwave png file here. Alternatively go to the folder and open the file

# from PIL import Image
# # Display the image in the default image viewer
# image = Image.open("sinewave.png")
# image.show()